In [4]:
import json
import pandas as pd
from tkinter import Tk, Label, Entry, Button, StringVar, messagebox, OptionMenu, Listbox, Scrollbar, END, font
import os
import datetime

# Class to manage expense tracking
class ExpenseTracker:
    def __init__(self):
        self.expenses = []
        self.load_data()

    def add_expense(self, amount, description, category):
        """Add an expense to the tracker."""
        expense = {
            'amount': amount,
            'description': description,
            'category': category,
            'date': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        self.expenses.append(expense)
        self.save_data()
        messagebox.showinfo("Success", "Expense added successfully!")

    def delete_expense(self, index):
        """Delete an expense from the tracker."""
        try:
            self.expenses.pop(index)
            self.save_data()
            messagebox.showinfo("Success", "Expense deleted successfully!")
        except IndexError:
            messagebox.showerror("Error", "Invalid selection. Please select a valid expense to delete.")

    def save_data(self):
        """Save expenses data to a JSON file."""
        with open('expenses.json', 'w') as file:
            json.dump(self.expenses, file, indent=4)
        print("Data saved successfully!")

    def load_data(self):
        """Load expenses data from a JSON file."""
        if os.path.exists('expenses.json'):
            with open('expenses.json', 'r') as file:
                self.expenses = json.load(file)
            print("Data loaded successfully!")
        else:
            self.expenses = []
            print("No previous data found, starting fresh.")

    def view_monthly_summary(self, month):
        """Display a monthly summary of expenses in a message box."""
        df = pd.DataFrame(self.expenses)
        if df.empty:
            messagebox.showinfo("No Data", "No expenses to show.")
            return
        
        df['date'] = pd.to_datetime(df['date'])
        monthly_expenses = df[df['date'].dt.to_period('M') == month]
        total = monthly_expenses['amount'].sum()

        if not monthly_expenses.empty:
            summary = f"Monthly Summary for {month}:\n\n"
            for idx, row in monthly_expenses.iterrows():
                summary += f"{row['date'].strftime('%Y-%m-%d %H:%M:%S')} - {row['amount']} - {row['description']} - {row['category']}\n"
            summary += f"\nTotal Expense: {total}"
            messagebox.showinfo("Monthly Summary", summary)
        else:
            messagebox.showinfo("No Data", f"There are no expenses recorded for the month: {month}.")

    def view_category_summary(self):
        """Display a summary of expenses by category in a message box."""
        df = pd.DataFrame(self.expenses)
        if df.empty:
            messagebox.showinfo("No Data", "No expenses to show.")
            return
        
        category_totals = df.groupby('category')['amount'].sum().reset_index()
        summary = "Expenses by Category:\n\n"
        for idx, row in category_totals.iterrows():
            summary += f"{row['category']}: {row['amount']}\n"
        
        messagebox.showinfo("Category Summary", summary)

# Function to add an expense using GUI input
def add_expense():
    try:
        amount = float(amount_var.get())
        description = description_var.get()
        category = category_var.get()
        if not description or not category:
            raise ValueError("Description and Category cannot be empty.")
        tracker.add_expense(amount, description, category)
        amount_var.set("")
        description_var.set("")
        refresh_expense_list()
    except ValueError as e:
        messagebox.showerror("Invalid Input", str(e))

# Function to delete selected expense from listbox
def delete_expense():
    selected_idx = expense_listbox.curselection()
    if selected_idx:
        tracker.delete_expense(selected_idx[0])
        refresh_expense_list()
    else:
        messagebox.showerror("Error", "Please select an expense to delete.")

# Function to refresh the listbox with current expenses
def refresh_expense_list():
    expense_listbox.delete(0, END)
    for expense in tracker.expenses:
        expense_listbox.insert(END, f"{expense['date']} - {expense['amount']} - {expense['description']} - {expense['category']}")

# Function to close the application
def close_app():
    root.destroy()

# Setting up the GUI with a unique theme
tracker = ExpenseTracker()
root = Tk()
root.title("Expense Tracker")

# Configure the window to dynamically match the screen size
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
root.geometry(f"{screen_width}x{screen_height}+0+0")

# Custom font settings
custom_font = font.Font(family="Helvetica", size=18, weight="bold")

amount_var = StringVar()
description_var = StringVar()
category_var = StringVar()

# Placing the listbox at the top left corner
expense_listbox = Listbox(root, width=50, height=20, font=custom_font, bg='#ecf0f1', fg='#2c3e50')
expense_listbox.grid(row=0, column=0, rowspan=8, padx=20, pady=20)

scrollbar = Scrollbar(root)
scrollbar.grid(row=0, column=1, rowspan=8, sticky='ns')
expense_listbox.config(yscrollcommand=scrollbar.set)
scrollbar.config(command=expense_listbox.yview)

Label(root, text="Amount:", font=custom_font, bg='#34495e', fg='white').grid(row=0, column=2, padx=20, pady=20, sticky="e")
Entry(root, textvariable=amount_var, font=custom_font).grid(row=0, column=3, padx=20, pady=20)

Label(root, text="Description:", font=custom_font, bg='#34495e', fg='white').grid(row=1, column=2, padx=20, pady=20, sticky="e")
Entry(root, textvariable=description_var, font=custom_font).grid(row=1, column=3, padx=20, pady=20)

Label(root, text="Category:", font=custom_font, bg='#34495e', fg='white').grid(row=2, column=2, padx=20, pady=20, sticky="e")
categories = ["Food", "Transportation", "Entertainment", "Utilities", "Others"]
category_var.set(categories[0])
OptionMenu(root, category_var, *categories).grid(row=2, column=3, padx=20, pady=20)

Button(root, text="Add Expense", font=custom_font, bg='#2ecc71', fg='white', command=add_expense).grid(row=3, column=2, columnspan=2, pady=20)
Button(root, text="Delete Selected Expense", font=custom_font, bg='#e74c3c', fg='white', command=delete_expense).grid(row=4, column=2, columnspan=2, pady=20)
Button(root, text="View Monthly Summary", font=custom_font, bg='#3498db', fg='white', command=lambda: tracker.view_monthly_summary("2024-08")).grid(row=5, column=2, columnspan=2, pady=20)
Button(root, text="View Category Summary", font=custom_font, bg='#9b59b6', fg='white', command=tracker.view_category_summary).grid(row=6, column=2, columnspan=2, pady=20)
Button(root, text="Close", font=custom_font, bg='#95a5a6', fg='white', command=close_app).grid(row=7, column=2, columnspan=2, pady=20)

refresh_expense_list()

# Bind the Escape key to exit full-screen mode
def exit_fullscreen(event=None):
    root.attributes("-fullscreen", False)

root.bind("<Escape>", exit_fullscreen)

root.mainloop()


Data loaded successfully!
Data saved successfully!
Data saved successfully!
Data saved successfully!
Data saved successfully!
Data saved successfully!
Data saved successfully!
Data saved successfully!
Data saved successfully!
Data saved successfully!
Data saved successfully!
